In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
# Bills the dental PAC cares about
bills = [
    ('HB 1027', 'SB 5159', 'Dental health aide therapists (midlevel provider)', 'oppose'),
    ('HB 1421', 'SB 5465', 'Concerning midlevel dental professionals', 'oppose'),
    ('HB 1441', '', 'Dental health services in tribal settings', 'oppose'),
    ('HB 2531',
     'SB 6318',
     'Concerning ownership, maintenance, and operation of an office within the practice of dentistry',
     'support'),
    ('HB 1514', '', 'Dental office support services', 'oppose'),
    ('HB 2517', 'SB 6275', 'Dental office support services', 'oppose'),
    ('HB 2819', 'SB 6404', 'Concerning dental licensure through completion of a residency program', 'support'),
    ('HB 1080',
     'SB 5010',
     'Restoring funding to the health professional loan repayment and scholarship fund',
     'support'),
    ('HB 2313', 'SB 6157', 'Raising the minimum legal smoking age to 21', 'support'),
    ('HB 1384',
     'SB 5606',
     'Modifying provisions related to licensing and scope of practice for dental professionals',
     'support'),
    ('HB 1002', '', 'Prohibiting unfair and deceptive dental insurance practices', 'support'),
    ('HB 1712', '', 'Concerning audit procedures to assist Medicaid providers', 'support'),
    ('HB 1852', '', 'Ensuring consumer choice through the health benefit exchange (pediatric)', 'support'),
    ('HB 2498',
     '',
     'Reforming the current prior authorization for dental services and supplies in medical assistance programs',
     'support'),
    ('HB 2499', '', 'Addressing fair insurance practices', 'support'),
    ('', 'SB 6648', 'Concerning benefits and exclusion within dental benefit coverage', 'support')
]

In [3]:
from sunlight import openstates
legislators = {}

for bill in bills:
    for bill_id in [bid for bid in [bill[0], bill[1]]]:
        if bill_id:
            detail = openstates.bill_detail('wa', '2015-2016', bill_id)
            if len(detail['votes']):
                # Question is whether vote corresponded to PAC position.
                # Other = other unless PAC position is other
                # Otherwise (heh), agreement/disagreement with PAC is what we want to capture
                for vote_type in [('yes_votes', 'support'), ('no_votes', 'oppose'), ('other_votes', 'other')]:
                    if vote_type[1] == 'other' and vote_type[1] != bill[3]:
                        category = 'other'
                    elif vote_type[1] != bill[3]:
                        category = 'oppose'
                    else:
                        category = 'support'
                    for bill_vote in detail['votes']:
                        for vote in bill_vote[vote_type[0]]:
                            key = vote['leg_id']
                            if key not in legislators:
                                legislators[key] = {
                                    'name': sponsor['name'],
                                    'sponsored': {
                                        'support': [],
                                        'oppose': []
                                    },
                                    'voted': {
                                        'support': [],
                                        'oppose': [],
                                        'other': []
                                    }
                                }
                            legislators[key]['voted'][category].append(bill_id)
            for sponsor in detail['sponsors']:
                key = sponsor['leg_id']
                if key not in legislators:
                    legislators[key] = {
                        'name': sponsor['name'],
                        'sponsored': {
                            'support': [],
                            'oppose': []
                        },
                        'voted': {
                            'support': [],
                            'oppose': [],
                            'other': []
                        }
                    }
                legislators[key]['sponsored'][bill[3]].append(bill_id)

ImportError: No module named sunlight

In [4]:
missing_legislators = []
for legislator in legislators:
    cmtes = ScrapedCommittee.objects.filter(committee_group='leg', name__icontains=legislators[legislator]['name'])
    if len(set([cmte.name for cmte in cmtes])) == 1:
        legislators[legislator]['committees'] = cmtes
    else:
        missing_legislators.append(legislators[legislator]['name'])
        legislators[legislator]['committees'] = []
print len(missing_legislators)
print '; '.join(missing_legislators)
print len(legislators)

NameError: name 'legislators' is not defined

In [5]:
def get_legislator_score(key):
    support = len(legislators[key]['sponsored']['support']) + len(legislators[key]['voted']['support'])
    oppose = len(legislators[key]['sponsored']['oppose']) + len(legislators[key]['voted']['oppose'])
    return support - oppose

correlation_data = {
    'donations': [],
    'scores': []
}
for legislator in sorted(
        legislators,
        key=lambda k: get_legislator_score(k),
        reverse=True):
    print '%s sponsored %d bills supported by the PAC (%s), and %d bills opposed by it (%s)' % (
        legislators[legislator]['name'],
        len(legislators[legislator]['sponsored']['support']),
        '; '.join(legislators[legislator]['sponsored']['support']),
        len(legislators[legislator]['sponsored']['oppose']),
        '; '.join(legislators[legislator]['sponsored']['oppose']))
    print '%s voted with the PAC on %d bills (%s), and against the PAC on %d bills (%s)' % (
        legislators[legislator]['name'],
        len(legislators[legislator]['voted']['support']),
        '; '.join(legislators[legislator]['voted']['support']),
        len(legislators[legislator]['voted']['oppose']),
        '; '.join(legislators[legislator]['voted']['oppose']))
    print '%s voted "other" on %d bills' % (
        legislators[legislator]['name'],
        len(legislators[legislator]['voted']['other']))
    print '%s scored %d' % (legislators[legislator]['name'], get_legislator_score(legislator))
    contribs = ScrapedContribution.objects.filter(filing_committee__in=legislators[legislator]['committees'])
    dental_contribs = contribs.filter(Q(donor__icontains='dental') | Q(donor__icontains='dentist'))
    # dental_contribs = contribs.filter(donor__icontains='dental pac')
    print '; '.join(set([d.donor for d in dental_contribs]))
    dental_contribs = dental_contribs.aggregate(
        Count('date'), Sum('amount'))
    if legislators[legislator]['committees']:
        correlation_data['scores'].append(get_legislator_score(legislator))
        if not dental_contribs or not dental_contribs['amount__sum']:
            correlation_data['donations'].append(0)
        else:
            correlation_data['donations'].append(float(dental_contribs['amount__sum']))
    if dental_contribs and dental_contribs['date__count']:
        print 'Dental organizations gave %d contributions totalling %s' % (
            dental_contribs['date__count'], locale.currency(dental_contribs['amount__sum'], grouping=True))
    elif not legislators[legislator]['committees']:
        print 'No fundraising committees for this legislator'
    else:
        print 'Unable to find any dental contributions to this legislator'
    print '-----'

NameError: name 'legislators' is not defined

In [6]:
# Correlation between scores and money?
from scipy.stats.stats import pearsonr

print pearsonr(correlation_data['scores'], correlation_data['donations'])

ImportError: No module named scipy.stats.stats

In [7]:
cmtes = ScrapedCommittee.objects.filter(committee_group__in=['caucus', 'party_state'])
for cmte in cmtes:
    print cmte.name

THE LEADERSHIP COUNCIL
HARRY TRUMAN FUND
HOUSE DEMO CAMP COMM
WA ST DEMO CENT COMM EXEMPT
WA ST DEMO CENT COMM NON EXEMPT
WA ST PROGRESSIVE CAUCUS
WA ST REPUB PARTY EXEMPT
7TH LEG DIST DEMO CENT COMM NON EXEMPT
REAGAN FUND
SENATE REPUB CAMP COMM
WA SENATE DEMO CAMP
WA ST REPUB PARTY NON EXEMPT
HOUSE REPUB ORG COMM
KENNEDY FUND


In [8]:
parties = ScrapedContribution.objects.filter(filing_committee__committee_group='leg').values(
    'filing_committee__party').annotate(count=Count('filing_committee__name'))
print parties
parties = ScrapedCommittee.objects.filter(committee_group='leg').values('party').annotate(count=Count('name'))
print parties

[{'count': 32, 'filing_committee__party': u'NO'}, {'count': 169, 'filing_committee__party': u'L'}, {'count': 34049, 'filing_committee__party': u'D'}, {'count': 27855, 'filing_committee__party': u'R'}, {'count': 3524, 'filing_committee__party': u'O'}]
[{'count': 8, 'party': u'NO'}, {'count': 2, 'party': u'N'}, {'count': 25, 'party': u'L'}, {'count': 141, 'party': u'R'}, {'count': 164, 'party': u'D'}, {'count': 16, 'party': u'O'}, {'count': 2, 'party': u'I'}]
